In [1]:
#Necesary imports.
import bitso
import pandas as pd
import numpy as np
from datetime import timezone
import datetime
from dateutil.tz import tzutc
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')
import plotly.offline
from plotly import graph_objs as go

In [2]:
#Opening my api key and secret.
with open('../key.txt') as f:
    a = f.readlines()

with open('../secret.txt') as p:
    b = p.readlines()

#Using the api secret and key to call the api.
API_SECRET = [i.split('\n') for i in a][0][0]
API_KEY = [i.split('\n') for i in b][0][0]
api = bitso.Api(API_KEY, API_SECRET)

In [16]:
'''
marker for last_scrap on 26-1-2020: 13337280, I save this marker so i can use it
the next time i'm scrapping
'''
#Defining the empyt lists of the elements we need from the api.
btc_mxn_date = []
btc_mxn_price = []
btc_mxn_volume = []
btc_mxn_hour = []
btc_mxn_side = []

#Marker than changes the time in the api calls.
mark = 13337280

#Datetime when the code is running
now = datetime.datetime.now(timezone.utc)

#While loop for scrapping the information we need
while 1:
    #limit of requests, the max we can get is 100
    limit = '100'
    #Calling the api trades, we use sort 'asc' so i't starts on the marker we want.
    trades = api.trades('btc_mxn', sort='asc', limit=limit, marker=mark)
    #Starting the counter so we don't raise the error of too many requests on the api.
    count = 0
    #printing the mark and the spaces, we need these so we know the last mark we use.
    print(mark)
    print('')
    print('')
    #if statement to break the while loop.
    if trades != []:
        #starting a for loop to append the elements we need from each trade to our lists
        for e in trades:
            #adding to the counter.
            count +=1
            # if statement so we just scrap elements before today.
            if e.created_at < now :
                #printing each trade so we can see that everythings is working fine.
                print(e)
                #scrapping the elements we need
                btc_mxn_price.append(e.price)
                btc_mxn_volume.append(e.amount)
                btc_mxn_hour.append(e.created_at.hour)
                btc_mxn_date.append(e.created_at.date())
                btc_mxn_side.append(e.maker_side)
        '''
        if statement that starts a timer when the count reachs 100, we can just get
        300 requests per minute so our timer waits 21 seconds before making the next
        100 requests. It also adds a thousand to our mark so it moves forward in time.
        '''
        if count == 100:
            time.sleep(21)
            count = 0
            mark = mark + 1000
            
    #else statement to break the loop.
    else:
        break
        

13302280


Trade(tid=13302284, price=161674.74, amount=0.00112064, maker_side=buy, created_at=2020-01-23 00:01:15+00:00)
Trade(tid=13302285, price=161757.27, amount=0.00012307, maker_side=sell, created_at=2020-01-23 00:01:15+00:00)
Trade(tid=13302290, price=161757.27, amount=0.00061821, maker_side=sell, created_at=2020-01-23 00:01:57+00:00)
Trade(tid=13302298, price=161757.27, amount=0.00037584, maker_side=sell, created_at=2020-01-23 00:02:35+00:00)
Trade(tid=13302301, price=161757.27, amount=0.00027201, maker_side=sell, created_at=2020-01-23 00:03:14+00:00)
Trade(tid=13302302, price=161757.27, amount=0.02202684, maker_side=sell, created_at=2020-01-23 00:03:19+00:00)
Trade(tid=13302310, price=161757.27, amount=0.00528570, maker_side=sell, created_at=2020-01-23 00:04:37+00:00)
Trade(tid=13302313, price=161757.27, amount=0.00024366, maker_side=sell, created_at=2020-01-23 00:04:54+00:00)
Trade(tid=13302315, price=161675.70, amount=0.00064934, maker_side=buy, created_at=2020-01-23 00:05:10

13303280


Trade(tid=13303292, price=160700.00, amount=0.03767717, maker_side=buy, created_at=2020-01-23 01:51:10+00:00)
Trade(tid=13303293, price=160680.00, amount=0.03984065, maker_side=buy, created_at=2020-01-23 01:51:10+00:00)
Trade(tid=13303294, price=160650.98, amount=0.00852919, maker_side=buy, created_at=2020-01-23 01:51:10+00:00)
Trade(tid=13303303, price=160660.29, amount=0.00144666, maker_side=sell, created_at=2020-01-23 01:51:16+00:00)
Trade(tid=13303304, price=160660.29, amount=0.09823686, maker_side=sell, created_at=2020-01-23 01:51:16+00:00)
Trade(tid=13303305, price=160660.29, amount=0.00003735, maker_side=sell, created_at=2020-01-23 01:51:17+00:00)
Trade(tid=13303306, price=160660.29, amount=0.00173682, maker_side=sell, created_at=2020-01-23 01:51:32+00:00)
Trade(tid=13303307, price=160660.29, amount=0.00013003, maker_side=sell, created_at=2020-01-23 01:51:32+00:00)
Trade(tid=13303308, price=160660.29, amount=0.01238419, maker_side=sell, created_at=2020-01-23 01:51:33+

13304280


Trade(tid=13304282, price=160597.95, amount=0.01305000, maker_side=buy, created_at=2020-01-23 03:42:59+00:00)
Trade(tid=13304287, price=160686.19, amount=0.00094557, maker_side=sell, created_at=2020-01-23 03:44:40+00:00)
Trade(tid=13304288, price=160686.19, amount=0.01443360, maker_side=sell, created_at=2020-01-23 03:44:45+00:00)
Trade(tid=13304292, price=160597.94, amount=0.01046970, maker_side=buy, created_at=2020-01-23 03:45:56+00:00)
Trade(tid=13304294, price=160686.18, amount=0.00010000, maker_side=sell, created_at=2020-01-23 03:46:30+00:00)
Trade(tid=13304295, price=160686.20, amount=0.00043178, maker_side=sell, created_at=2020-01-23 03:46:30+00:00)
Trade(tid=13304297, price=160686.20, amount=0.02924956, maker_side=sell, created_at=2020-01-23 03:46:53+00:00)
Trade(tid=13304301, price=160597.94, amount=0.00311337, maker_side=buy, created_at=2020-01-23 03:47:28+00:00)
Trade(tid=13304302, price=160597.95, amount=0.00094013, maker_side=buy, created_at=2020-01-23 03:47:57+0

13305280


Trade(tid=13305292, price=160088.83, amount=0.00124931, maker_side=sell, created_at=2020-01-23 06:23:11+00:00)
Trade(tid=13305293, price=160006.70, amount=0.00100975, maker_side=buy, created_at=2020-01-23 06:23:34+00:00)
Trade(tid=13305294, price=160006.70, amount=0.21578874, maker_side=buy, created_at=2020-01-23 06:23:34+00:00)
Trade(tid=13305299, price=159832.22, amount=0.02189796, maker_side=sell, created_at=2020-01-23 06:25:03+00:00)
Trade(tid=13305300, price=159741.97, amount=0.00630106, maker_side=buy, created_at=2020-01-23 06:25:18+00:00)
Trade(tid=13305301, price=159832.22, amount=0.00375394, maker_side=sell, created_at=2020-01-23 06:25:19+00:00)
Trade(tid=13305303, price=159741.97, amount=0.00173501, maker_side=buy, created_at=2020-01-23 06:26:32+00:00)
Trade(tid=13305304, price=159741.97, amount=0.02528446, maker_side=buy, created_at=2020-01-23 06:26:41+00:00)
Trade(tid=13305307, price=159741.97, amount=0.02712814, maker_side=buy, created_at=2020-01-23 06:28:21+00:

13306280


Trade(tid=13306281, price=158000.99, amount=0.85538416, maker_side=buy, created_at=2020-01-23 10:54:06+00:00)
Trade(tid=13306286, price=158091.72, amount=0.00267263, maker_side=sell, created_at=2020-01-23 10:57:20+00:00)
Trade(tid=13306288, price=158091.72, amount=0.00004061, maker_side=sell, created_at=2020-01-23 10:58:04+00:00)
Trade(tid=13306296, price=158010.00, amount=0.00632872, maker_side=buy, created_at=2020-01-23 11:04:27+00:00)
Trade(tid=13306297, price=158000.99, amount=0.14504887, maker_side=buy, created_at=2020-01-23 11:04:27+00:00)
Trade(tid=13306301, price=158010.00, amount=0.00316436, maker_side=buy, created_at=2020-01-23 11:08:30+00:00)
Trade(tid=13306302, price=158010.00, amount=0.00316436, maker_side=buy, created_at=2020-01-23 11:08:30+00:00)
Trade(tid=13306303, price=158000.99, amount=0.21607046, maker_side=buy, created_at=2020-01-23 11:08:30+00:00)
Trade(tid=13306305, price=158000.99, amount=1.02204637, maker_side=buy, created_at=2020-01-23 11:08:38+00:0

...

13335280


Trade(tid=13335284, price=160955.12, amount=0.00194711, maker_side=buy, created_at=2020-01-26 18:47:56+00:00)
Trade(tid=13335287, price=160955.00, amount=0.00317962, maker_side=buy, created_at=2020-01-26 18:48:18+00:00)
Trade(tid=13335290, price=161349.90, amount=0.00030989, maker_side=sell, created_at=2020-01-26 18:49:17+00:00)
Trade(tid=13335297, price=160955.59, amount=0.00192551, maker_side=buy, created_at=2020-01-26 18:49:31+00:00)
Trade(tid=13335298, price=160955.65, amount=0.00012508, maker_side=buy, created_at=2020-01-26 18:49:42+00:00)
Trade(tid=13335303, price=161200.00, amount=0.00192982, maker_side=buy, created_at=2020-01-26 18:50:15+00:00)
Trade(tid=13335306, price=161200.00, amount=0.00193707, maker_side=buy, created_at=2020-01-26 18:50:25+00:00)
Trade(tid=13335309, price=161349.89, amount=0.00061977, maker_side=sell, created_at=2020-01-26 18:50:35+00:00)
Trade(tid=13335312, price=161200.01, amount=0.00194188, maker_side=buy, created_at=2020-01-26 18:50:37+00:0

13336280


Trade(tid=13336282, price=160011.75, amount=0.01344646, maker_side=buy, created_at=2020-01-26 21:16:57+00:00)
Trade(tid=13336283, price=160011.90, amount=0.01258085, maker_side=buy, created_at=2020-01-26 21:17:07+00:00)
Trade(tid=13336284, price=160012.13, amount=0.00629042, maker_side=buy, created_at=2020-01-26 21:17:20+00:00)
Trade(tid=13336288, price=160999.97, amount=0.00018919, maker_side=sell, created_at=2020-01-26 21:18:55+00:00)
Trade(tid=13336292, price=160999.97, amount=0.00403389, maker_side=sell, created_at=2020-01-26 21:21:18+00:00)
Trade(tid=13336296, price=160800.00, amount=0.00310945, maker_side=sell, created_at=2020-01-26 21:22:01+00:00)
Trade(tid=13336298, price=160015.07, amount=0.08346807, maker_side=buy, created_at=2020-01-26 21:22:31+00:00)
Trade(tid=13336299, price=160800.00, amount=0.00070203, maker_side=sell, created_at=2020-01-26 21:23:05+00:00)
Trade(tid=13336300, price=160999.64, amount=0.00240444, maker_side=sell, created_at=2020-01-26 21:23:05+0

13337280


Trade(tid=13337283, price=160350.59, amount=0.00060727, maker_side=buy, created_at=2020-01-26 23:50:05+00:00)
Trade(tid=13337284, price=160351.01, amount=0.00941568, maker_side=buy, created_at=2020-01-26 23:50:17+00:00)
Trade(tid=13337289, price=160352.12, amount=0.00005650, maker_side=buy, created_at=2020-01-26 23:50:47+00:00)
Trade(tid=13337290, price=161300.63, amount=0.02254600, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337291, price=161300.63, amount=0.02760112, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337292, price=161300.64, amount=0.05296758, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337293, price=161300.64, amount=0.00030998, maker_side=sell, created_at=2020-01-26 23:51:42+00:00)
Trade(tid=13337299, price=161300.64, amount=0.01474272, maker_side=sell, created_at=2020-01-26 23:53:11+00:00)
Trade(tid=13337300, price=161300.64, amount=0.00012399, maker_side=sell, created_at=2020-01-26 23:53:12+

13338280


Trade(tid=13338295, price=162497.51, amount=0.02338497, maker_side=sell, created_at=2020-01-27 01:02:38+00:00)
Trade(tid=13338301, price=162497.05, amount=0.00309535, maker_side=sell, created_at=2020-01-27 01:03:57+00:00)
Trade(tid=13338302, price=162002.22, amount=0.00062132, maker_side=buy, created_at=2020-01-27 01:04:05+00:00)
Trade(tid=13338309, price=162496.71, amount=0.01720686, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338310, price=162496.71, amount=0.01811248, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338311, price=162496.72, amount=0.11237595, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338312, price=162496.88, amount=0.00363155, maker_side=sell, created_at=2020-01-27 01:04:55+00:00)
Trade(tid=13338313, price=162496.89, amount=0.01975351, maker_side=sell, created_at=2020-01-27 01:04:55+00:00)
Trade(tid=13338314, price=162002.64, amount=0.00237024, maker_side=buy, created_at=2020-01-27 01:05:06

13339280




In [17]:
# creating a new data frame.
df = pd.DataFrame(columns=['price', 'volume', 'hour', 'date', 'side'])
# using the lists we got from the while loop to fill the dataframe.
df.price = btc_mxn_price
df.volume = btc_mxn_volume
df.hour = btc_mxn_hour
df.date = btc_mxn_date
df.side = btc_mxn_side
df.date = df.date.astype('datetime64')
#creating new columns with the month and day of each row.
df['month'] = [i.month for i in df.date]
df['day'] = [i.day for i in df.date]

In [18]:
#Creating a function that needs a data frame and a number(month) for input.
def extraction(df, m):
    #variable that get's just the row that matches the same month
    f_df = df.loc[df['month'] == m]
    #variable that stores the year
    y = datetime.datetime.today().year
    #variable that creates a list with the numbers of days of the month
    days = list(f_df['day'].unique())
    #varible that creates a empty dictionary
    results = {}
    #list with the names of the keys in the dictionary that the function returns.
    nombres = ['date', '24h_avg', '12h_avg', 'min', 'max', 'volume', 'm_side']
    
    #for loop that creates new keys in the empty dictionary with a empty list as values.
    for i in nombres:
        results[i] = []
    
    #for that loops on each day in the list of the days of the month.
    for e in days:
        #variable that get's just the rows that match the day in the for loop.
        ndf = f_df.loc[f_df.day == e]
        #appending the results in each dictionary key.
        results['date'].append(datetime.date(y, m, e))
        results['24h_avg'].append(ndf.price.mean())
        results['12h_avg'].append(ndf.price.loc[ndf.hour<12].mean())
        results['max'].append(ndf.price.max())
        results['min'].append(ndf.price.min())
        results['volume'].append(ndf.volume.sum())
        weight = ndf.side.value_counts().sum()
        buy = ndf.side.value_counts()['buy']/weight
        sell = ndf.side.value_counts()['sell']/weight
        if buy>sell:
            results['m_side'].append('buy')
        else:
            results['m_side'].append('sell')
            
    #returns the dictionary with the appended elements
    return results

In [19]:
#creating a variable that stores the month we are in.
month = datetime.datetime.today().month
#creating a new data frame using the dictionary from the above function and the month
df_new = pd.DataFrame.from_dict(extraction(df, month))

In [20]:
#Reading our csv file where we store the past data
history = pd.read_csv('../csv_files/historic_data.csv')

In [21]:
btc_df = pd.concat([history, df_new], ignore_index=True)

In [22]:
btc_df.drop_duplicates(inplace=True)

In [23]:
btc_df.drop(btc_df.tail(1).index,inplace=True)

In [24]:
btc_df.to_csv('../csv_files/historic_data.csv', index=False)

In [25]:
btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

In [26]:
df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)

In [27]:
change = []
for i in range(1,len(df_dum)):
    if df_dum.sell.iloc[i] != df_dum.sell.iloc[i-1]:
        change.append(df_dum.date.iloc[i])

In [28]:
#Selecting the columns we need for fbprophet
btc_24h_avg = btc_df[['date', '24h_avg']].copy()
#Changing the columns names for fbprophet to accept it.
btc_24h_avg.rename(columns={'date': 'ds', '24h_avg': 'y'}, inplace=True)

In [29]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [30]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [31]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    df.dropna(inplace=True)
    
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [32]:
train_df = btc_24h_avg.copy().set_index('ds')

In [33]:
train_df['y'], lambda_prophet = boxcox(train_df['y'])
train_df.reset_index(inplace=True)

In [34]:
m2 = Prophet(changepoints=change, seasonality_mode='multiplicative')
m2.fit(train_df)
future2 = m2.make_future_dataframe(periods=15)
forecast2 = m2.predict(future2)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [35]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column],
                                       lambda_prophet)

In [36]:
cmp_df2 = make_comparison_dataframe(btc_24h_avg, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, 15).items():
     print(err_name, err_value)

MAPE 8.613481266075071
MAE 13950.221083035216


In [37]:


def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Zona de compra', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="green"))
    upper_bound = create_go('Máximo esperado', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="red"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Venta sugerida', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Precio promedio del día', 'y', num_values,
                       marker=dict(color="red"))
    
    
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='MXN'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='btc_mxn.html', auto_open=False)

show_forecast(cmp_df2, 15, 2123, 'BTC')

